In [3]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [5]:
# 1. load your full data
df = pd.read_csv('../data/training.csv')

# 2. take a reproducible random sample of 100 rows
df100 = df.sample(n=100, random_state=42).reset_index(drop=True)

# 3. drop the existing label column
df100_nolabel = df100[["id","sentence"]]
df100_nolabel["label"] = ""        # ← empty column for your teammates

# 4. split into four 25-row chunks
chunks = np.array_split(df100_nolabel, 4)

# 5. write each chunk
for i, chunk in enumerate(chunks, start=1):
    chunk.to_csv(f"group_member_{i}_to_label.csv", index=False)

# Now you have:
#   group_member_1_to_label.csv
#   group_member_2_to_label.csv
#   group_member_3_to_label.csv
#   group_member_4_to_label.csv
#
# Each file contains only “id” and “sentence” and exactly 25 rows.

/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
# AFTER your teammates label their 25 sentences, you’ll get back four CSVs of form:
#   id,sentence,label
# with 25 rows each. To merge them back into one “100-row with labels” CSV:
import glob
files = glob.glob("group_member_*_labeled.csv")  # wherever they save them

labeled = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

# (optional) re-order to match the original sampled order
labeled = labeled.set_index("id").loc[df100["id"]].reset_index()

labeled.to_csv("100_labeled_by_team.csv", index=False)